In [3]:
#!/usr/bin/env python3
"""
Database migration to add who_referred column to paid_cases table
Fixed for modern SQLAlchemy versions

Usage:
    python migrate_add_who_referred.py
"""

import os
import sys
from sqlalchemy import text, inspect

# Add the project root to Python path
sys.path.insert(0, os.path.dirname(os.path.abspath(r'C:\Users\VasilisKottas\Documents\Intranet_v2\app')))

from app import create_app
from app.models import db

def check_column_exists():
    """Check if who_referred column already exists using SQLAlchemy inspector"""
    try:
        inspector = inspect(db.engine)
        columns = inspector.get_columns('paid_cases')
        column_names = [col['name'] for col in columns]
        return 'who_referred' in column_names
    except Exception as e:
        print(f"⚠️ Error checking columns: {e}")
        return False

def run_migration():
    """Run the migration to add who_referred column"""
    
    print("🔄 Starting database migration...")
    
    # Determine environment
    config_name = 'production' if os.environ.get('RENDER') else 'development'
    print(f"📋 Environment: {config_name}")
    
    # Create app and establish database connection
    app = create_app(config_name)
    
    with app.app_context():
        try:
            print(f"🔗 Database: {db.engine.url.drivername}")
            
            # Check if column already exists
            if check_column_exists():
                print("✅ Column 'who_referred' already exists in paid_cases table")
                print("🎉 Migration already completed!")
                return
            
            print("➕ Adding 'who_referred' column to paid_cases table...")
            
            # Modern SQLAlchemy approach - use connection context
            with db.engine.begin() as connection:
                connection.execute(text("""
                    ALTER TABLE paid_cases ADD COLUMN who_referred VARCHAR(200)
                """))
            
            print("✅ Successfully added 'who_referred' column!")
            
            # Verify the column was added
            if check_column_exists():
                print("✅ Migration verification successful!")
                print("🎉 Migration completed successfully!")
            else:
                print("❌ Migration verification failed!")
                
        except Exception as e:
            print(f"❌ Error running migration: {e}")
            print(f"📋 Error type: {type(e).__name__}")
            raise

if __name__ == '__main__':
    try:
        run_migration()
    except KeyboardInterrupt:
        print("\n⛔ Migration cancelled by user")
        sys.exit(1)
    except Exception as e:
        print(f"\n💥 Migration failed: {e}")
        sys.exit(1)

🔄 Starting database migration...
📋 Environment: development
✓ Loaded development configuration
✓ Template folder: c:\Users\VasilisKottas\Documents\Intranet_v2\templates
✓ Static folder: c:\Users\VasilisKottas\Documents\Intranet_v2\static
✓ Configured normal session support (development mode)
✓ Configured CORS for origins: ['https://windsorhillmortgages.co.uk', 'https://windsorhillmortgages.co.uk/winsuite', 'https://windsorhillmortgages.co.uk/dashboard', 'https://windsorhillmortgages.co.uk/sales-dashboard', 'https://www.windsorhillmortgages.co.uk', 'https://www.windsorhillmortgages.co.uk/winsuite', 'https://www.windsorhillmortgages.co.uk/dashboard', 'https://www.windsorhillmortgages.co.uk/sales-dashboard', 'https://sales-dashboard-5g5x.onrender.com']
 All middlewares configured successfully
 All controllers registered successfully
🔗 Database: sqlite
➕ Adding 'who_referred' column to paid_cases table...
✅ Successfully added 'who_referred' column!
✅ Migration verification successful!
🎉 Mi